# Python: Introduction to Machine Learning

## Import Libraries 

## Import Data 

## Exploratory Data Analysis (EDA)

## Model Development

### Data Preparation

### Model Creation

### Model Evaluation

## Hypterparamter Tuning

## Model Testing

## Considerations 
 - Test out different algorithms -> Random Forest for Regression 
 - Iterate over the feature selection process 
 - Feature Engineering: Develop your own features from the available data 
